<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-final/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 49.4 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
11.8


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu118
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone -b sg-experiment https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric, temporal_dataset_split

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 178 (delta 10), reused 5 (delta 5), pack-reused 152
Receiving objects: 100% (178/178), 138.07 KiB | 1.44 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [5]:
!pip install -U torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 40.2 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=6420bf9de13a9729e06a349ea26e4378d1c2071266ca179cb5f12561338fccdc
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires pyth

In [6]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# Temporal Datasets
# from torch_geometric_temporal.dataset import METRLADatasetLoader
# loader = METRLADatasetLoader()
# dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
# iterator = iter(dataset)
# print("METRLADatasetLoader", next(iterator))
# print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset(lags=8) # consistent with chickenpox paper
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
# dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
# iterator = iter(dataset)
# print("WikiMathsDatasetLoader", next(iterator))
# print("WikiMathsDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))


# Graph summaries
# METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
# ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
# WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])

ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])


In [16]:
# Run Chickenpox Dataset

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() # clear cuda cache

from torch_geometric_temporal.signal import temporal_signal_split
train_ratio = 0.8
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=train_ratio)
offset = int(dataset.snapshot_count * train_ratio) # starting index for test set

in_dim = dataset[0].num_node_features # 8 treat lagged inputs as node features
out_dim = 1
num_nodes = dataset[0].num_nodes # 1068
timesteps_to_predict = 10 # 10, 20, 40 week forecast horizon
epochs = 200
lrate = 0.0001
wdecay = 0.001
save_path = "store/checkpoint"

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

model.train()

def prepare_n_period_y(dataset):

    res = []
    for data in dataset:
        res.append(data.y)
    res = torch.stack(res, dim=0)

    return res

y_all = prepare_n_period_y(dataset)
training_curve_dict = {"epoch_train_loss": []}

for epoch in tqdm(range(epochs)):
    train_loss = []
    t1 = time.time()

    for i, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        if timesteps_to_predict == 1:
            y_train = snapshot.y.to(device)
        else:
            y_train = y_all[i : i + timesteps_to_predict,:].to(device)

        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mse(pred, y_train, 0.0) # mean squared error for loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss.append(loss.item())

    mtrainloss = np.mean(train_loss)
    training_curve_dict['epoch_train_loss'].append(mtrainloss)
    print(f"training loss: {mtrainloss}")


# eval
model.eval()
loss = 0
for i, snapshot in enumerate(test_dataset):
    if i + timesteps_to_predict > test_dataset.snapshot_count:
        break

    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)

    if timesteps_to_predict == 1:
        y_test = snapshot.y.to(device)
    else:
        y_test = y_all[offset+i : offset+i + timesteps_to_predict,:].to(device)


    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr).squeeze()
    loss += masked_mse(pred, y_test, 0.0) # mean squared error as loss

loss = loss / (i+1)
loss = loss.item()
print("Test MSE Loss: {:.4f}".format(loss))


# store training loss
pd.DataFrame(training_curve_dict).to_csv("training_curve.csv")

from google.colab import files
files.download('training_curve.csv')


start training...


  0%|          | 1/200 [00:20<1:07:57, 20.49s/it]

training loss: 0.966555262279765


  1%|          | 2/200 [00:40<1:05:49, 19.95s/it]

training loss: 0.9517792885350745


...

training loss: 0.803408170128014
Average Training Time: nan secs/epoch



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Test MSE Loss: 1.0971


In [ ]:
training_curve_dict["epoch_train_loss"]

In [ ]:
!nvidia-smi

In [ ]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()